In [68]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [69]:
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X.drop(['home.dest','body','boat','name','cabin','ticket'], axis=1, inplace=True)
X = pd.get_dummies(X,columns=['sex','embarked','pclass'])
X.dropna(inplace=True)
y = y[X.index]
# y[(X['sex_female']==1) & (X['pclass']==3.0)].value_counts()
X

,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,pclass_1.0,pclass_2.0,pclass_3.0
0,29.0000,0.0,0.0,211.3375,1,0,0,0,1,1,0,0
1,0.9167,1.0,2.0,151.5500,0,1,0,0,1,1,0,0
2,2.0000,1.0,2.0,151.5500,1,0,0,0,1,1,0,0
3,30.0000,1.0,2.0,151.5500,0,1,0,0,1,1,0,0
4,25.0000,1.0,2.0,151.5500,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1301,45.5000,0.0,0.0,7.2250,0,1,1,0,0,0,0,1
1304,14.5000,1.0,0.0,14.4542,1,0,1,0,0,0,0,1
1306,26.5000,0.0,0.0,7.2250,0,1,1,0,0,0,0,1
1307,27.0000,0.0,0.0,7.2250,0,1,1,0,0,0,0,1


In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
X_train

,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,pclass_1.0,pclass_2.0,pclass_3.0
56,36.0,1.0,2.0,120.0000,0,1,0,0,1,1,0,0
514,2.0,1.0,1.0,26.0000,0,1,0,0,1,0,1,0
685,21.0,0.0,0.0,16.1000,0,1,0,0,1,0,0,1
866,2.0,0.0,1.0,12.2875,1,0,0,0,1,0,0,1
187,16.0,0.0,1.0,39.4000,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
35,45.0,0.0,0.0,262.3750,1,0,1,0,0,1,0,0
329,36.0,1.0,0.0,26.0000,1,0,0,0,1,0,1,0
132,24.0,0.0,0.0,79.2000,0,1,1,0,0,1,0,0
676,18.0,0.0,0.0,7.7500,0,1,0,0,1,0,0,1


# Desicion tree

In [71]:
tree_clf = DecisionTreeClassifier(max_depth=4)
tree_clf.fit(X_train, y_train)

tree_clf.score(X_test, y_test)


0.7990430622009569

# SVM, LogisticRegressor

In [72]:
scaller = StandardScaler()
X_train1 = scaller.fit_transform(X_train)
X_test1 = scaller.transform(X_test)

svm_clf = SVC(gamma='auto')
# svm_clf.fit(X_train1, y_train)

log_clf = LogisticRegression(C=0.001)
log_clf.fit(X_train1, y_train)
log_clf.score(X_test1,y_test)

0.7464114832535885

In [73]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest_clf = RandomForestClassifier(n_estimators=100, random_state=55)
forest_clf.fit(X_train1, y_train)

svm_scores = cross_val_score(svm_clf, X_train1, y_train, cv=10)
svm_scores.mean()

0.8001864601262192

In [82]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(gamma='auto', probability=True)


voting_clf = VotingClassifier(
            estimators=[('lr', log_clf), ('svc', svm_clf), ('rf', rnd_clf)],
            voting='soft')
voting_clf.fit(X_train, y_train)

/home/dmytro/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('svc', SVC(gamma='auto', probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft')

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train1, y_train)
    y_pred = clf.predict(X_test1)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))